# Doc2Vec

Word2Vecの応用で、ドキュメント(文書)に適用した、Doc2Vecを紹介する。

Word2Vecで行ったものと同様に、Doc2Vecで同じドキュメント(文書)を学習し、ドキュメント間で似ているものを探し出す

手順

- 事前準備
- 学習する
- 似た文書を見つける


In [1]:
import urllib.request
import base64

out = []
text = ""

url = "https://hit-u-data-text-processing.herokuapp.com/data/20210502-news-text.txt"
auth_str = base64.b64encode(b"reader:hit-u")
req = urllib.request.Request(url,
                            headers={"Authorization": "Basic " + auth_str.decode("utf-8")})
with urllib.request.urlopen(req) as req:
    text = req.read().decode("utf-8")
    for line in text.split("\n"):
        if line.strip() == "===":
            if text:
                out.append(text)
            text = ""
        else:
            text += line
url = "https://hit-u-data-text-processing.herokuapp.com/data/20210612-news-text.txt"
auth_str = base64.b64encode(b"reader:hit-u")
req = urllib.request.Request(url,
                            headers={"Authorization": "Basic " + auth_str.decode("utf-8")})
with urllib.request.urlopen(req) as req:
    text = req.read().decode("utf-8")
    for line in text.split("\n"):
        if line.strip() == "===":
            if text:
                out.append(text)
            text = ""
        else:
            text += line

In [2]:
# out[1]

In [ ]:
# Colabで実行している場合
# pip install janome

In [3]:
from janome.tokenizer import Tokenizer
from janome.tokenfilter import TokenFilter
from janome.tokenfilter import CompoundNounFilter
from janome.tokenfilter import POSKeepFilter
from janome.tokenfilter import LowerCaseFilter
from janome.charfilter import UnicodeNormalizeCharFilter
from janome.analyzer import Analyzer

In [4]:
class StopWordFilter(TokenFilter):
    def __init__(self, words):
        self.stop_words = words
    
    def apply(self, tokens):
        for token in tokens:
            if token.surface not in self.stop_words:
                yield token

In [5]:
import urllib.request
import base64
url = "https://hit-u-data-text-processing.herokuapp.com/data/stopwords.txt"
auth_str = base64.b64encode(b"reader:hit-u")
req = urllib.request.Request(url,
                            headers={"Authorization": "Basic " + auth_str.decode("utf-8")})

stop_words = []
with urllib.request.urlopen(req) as req:
    lines = req.read().decode("utf-8")
    for line in lines.split("\n"):
        if line.strip():
            stop_words.append(line.strip())
stop_words

['大学', '一橋大学', '===', 'こと', 'the', 'ため', 'よう', 'of', '(', ')', '様']

In [6]:
stop_word_filter = StopWordFilter(stop_words)

In [7]:
token_filters = [CompoundNounFilter(),
                POSKeepFilter(["名詞", "動詞", "形容詞"]),
                LowerCaseFilter(),
                stop_word_filter]

In [8]:
char_filters = [UnicodeNormalizeCharFilter()]
tokenizer = Tokenizer()
analyzer = Analyzer(char_filters=char_filters, 
                    tokenizer=tokenizer, 
                    token_filters=token_filters)

In [9]:
docs_words = {}
for i, text in enumerate(out):
    docs_words[i] = []
    for token in analyzer.analyze(text):
        docs_words[i].append(token.base_form)

In [10]:
len(docs_words)

34

In [11]:
type(docs_words[0])

list

In [12]:
docs_words[12][:10]

['科研費',
 '本学',
 '新規採択率全国1位',
 '2021年1月6日',
 '令和2年度科学研究費助成事業',
 '科研費*(補助金',
 '基金分)〕',
 '配分状況',
 '2020年12月25日(金)',
 '文部科学省']

In [13]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [14]:
def get_sentences(docs_words):
    sentences = []
    for k, text in docs_words.items():
        sentences.append(TaggedDocument(words=text, tags=[str(k)]))
    return sentences

In [15]:
sentences = get_sentences(docs_words)

In [16]:
sentences[15]

TaggedDocument(words=['経済研究所長', '渡部敏明教授', 'ベイズ統計学', '国際学会', '部会chair-elect', '(次期会長)', '選出', 'する', 'れる', '2020年12月2日経済研究所長', '渡部敏明教授', 'ベイズ統計学', '国際学会international', 'society', 'for', 'bayesian', 'analysis', '(isba)', '部会economics,', 'finance,', 'and', 'business', '(efab)', 'chair-elect', '(次期会長)', '選出', 'する', 'れる', '2021年1月', '1年間', 'chair-elect', 'chair', '補佐', 'する', '後', '2022年1月', '1年間chair', '務める'], tags=['15'])

In [17]:
model = Doc2Vec(sentences, dm=1, vector_size=50, window=5, alpha=0.025,
        min_alpha=0.025, min_count=2, sample=1e-6)

In [18]:
def train_model(model, sentences, num_epoch=20):
    print('\n訓練開始')
    for epoch in range(num_epoch):
        print(f"Epoch: {epoch + 1}")
        model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)
        model.alpha -= (0.025 - 0.0001) / 19
        model.min_alpha = model.alpha
    return model

In [19]:
fit_model = train_model(model, sentences, 20)


訓練開始
Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
Epoch: 6
Epoch: 7
Epoch: 8
Epoch: 9
Epoch: 10
Epoch: 11
Epoch: 12
Epoch: 13
Epoch: 14
Epoch: 15
Epoch: 16
Epoch: 17
Epoch: 18
Epoch: 19
Epoch: 20


In [20]:
# Jupyter labでPython 3.9 gensim 4.2.0 を使っている場合
fit_model.dv.most_similar("0", topn=5)

[('19', 0.9999522566795349),
 ('17', 0.999922513961792),
 ('32', 0.9999180436134338),
 ('13', 0.9999134540557861),
 ('16', 0.9998354315757751)]

In [74]:
# Colabでgensim3.6.0を使っている場合
fit_model.docvecs.most_similar("0", topn=5)

[('17', 0.9999710321426392),
 ('32', 0.9997791647911072),
 ('13', 0.9997351169586182),
 ('19', 0.9996321201324463),
 ('14', 0.9980309009552002)]

In [21]:
for k, words in docs_words.items():
    print(f"{k} : {words[:5]}")

0 : ['ソニー', 'パナソニック', '富士通', '資生堂', '共同']
1 : ['令和2年度学位記授与式', '挙行', 'する', '2021年4月1日新型コロナウイルス感染症拡大防止', '学部別']
2 : ['cambridge', 'economic', 'history', 'modern', 'world(stephen']
3 : ['令和3年度入学式', '挙行', 'する', '2021年4月15日新型コロナウイルス感染症拡大防止', '学部別']
4 : ['学業優秀学生(在学生)', '決定', 'する', '2021年4月9日', '令和2年度一年間']
5 : ['令和2年度学位記授与式', '挙行', 'する', '2021年4月1日新型コロナウイルス感染症拡大防止', '学部別']
6 : ['53回内藤章記念賞論文入選者', '表彰式', '行う', '2021年4月1日2021年3月19日(金)、学位記授与式当日', '法人本部棟7階中会議室']
7 : ['学業優秀学生(卒業生)', '表彰', '行う', 'れる', '2021年3月22日学業優秀学生(卒業生)']
8 : ['知的財産共同研究成果', '3件目', '特許', '取得', 'する']
9 : ['独立行政法人日本学生支援機構', '発行', 'する', 'ソーシャルボンド', '投資']
10 : ['四大学連合ポストコロナ社会コンソーシアム', '覚書', '締結', 'する', '2021年2月1日東京医科歯科大学']
11 : ['令和2年司法試験', '結果', '2021年1月28日法科大学院修了者', '対象', 'する']
12 : ['科研費', '本学', '新規採択率全国1位', '2021年1月6日', '令和2年度科学研究費助成事業']
13 : ['公開講座開催案内2020年度一橋大学公開講座', '裁判員裁判', 'いま', '裁判員裁判', '重大']
14 : ['世界', '大学連合', 'u7+」', 'オンラインサミット', '中野学長']
15 : ['経済研究所長', '渡部敏明教授', 'ベイズ統計学', '国際学会', '部会chair-elect']
16 : ['中野聡学長', '世界', 'ラ

In [22]:
# Jupyter labでPython 3.9 gensim 4.2.0 を使っている場合
def doc2vec_similar(id_, topn=5):
    print(f"{id_}  -- {docs_words[int(id_)][:5]})")
    similars = fit_model.dv.most_similar(id_, topn=topn)
    for similar_id, score in similars:
        print(f"{similar_id} : {score} -- {docs_words[int(similar_id)][:5]})")

In [77]:
# Colabでgensim3.6.0を使っている場合
def doc2vec_similar(id_, topn=5):
    print(f"{id_}  -- {docs_words[int(id_)][:5]})")
    similars = fit_model.docvecs.most_similar(id_, topn=topn)
    for similar_id, score in similars:
        print(f"{similar_id} : {score} -- {docs_words[int(similar_id)][:5]})")

In [23]:
doc2vec_similar("19")

19  -- ['ソニー', 'パナソニック', '富士通', '資生堂', '共同'])
0 : 0.9999523162841797 -- ['ソニー', 'パナソニック', '富士通', '資生堂', '共同'])
17 : 0.9999119639396667 -- ['関西', '中部合同アカデミア', '2020年度一橋大学関西', '中部合同アカデミア', '局面'])
13 : 0.9998791217803955 -- ['公開講座開催案内2020年度一橋大学公開講座', '裁判員裁判', 'いま', '裁判員裁判', '重大'])
32 : 0.9998767375946045 -- ['公開講座開催案内2020年度一橋大学公開講座', '裁判員裁判', 'いま', '裁判員裁判', '重大'])
16 : 0.9998028874397278 -- ['中野聡学長', '世界', 'ライブ配信', 'する', 'れる'])


In [24]:
doc2vec_similar("3")

3  -- ['令和3年度入学式', '挙行', 'する', '2021年4月15日新型コロナウイルス感染症拡大防止', '学部別'])
16 : 0.999011754989624 -- ['中野聡学長', '世界', 'ライブ配信', 'する', 'れる'])
13 : 0.9989255666732788 -- ['公開講座開催案内2020年度一橋大学公開講座', '裁判員裁判', 'いま', '裁判員裁判', '重大'])
17 : 0.9989242553710938 -- ['関西', '中部合同アカデミア', '2020年度一橋大学関西', '中部合同アカデミア', '局面'])
27 : 0.9988905191421509 -- ['知的財産共同研究成果', '3件目', '特許', '取得', 'する'])
32 : 0.9988779425621033 -- ['公開講座開催案内2020年度一橋大学公開講座', '裁判員裁判', 'いま', '裁判員裁判', '重大'])


In [25]:
doc2vec_similar("31")

31  -- ['科研費', '本学', '新規採択率全国1位', '2021年1月6日', '令和2年度科学研究費助成事業'])
17 : 0.9995192885398865 -- ['関西', '中部合同アカデミア', '2020年度一橋大学関西', '中部合同アカデミア', '局面'])
0 : 0.9995107650756836 -- ['ソニー', 'パナソニック', '富士通', '資生堂', '共同'])
13 : 0.9994748830795288 -- ['公開講座開催案内2020年度一橋大学公開講座', '裁判員裁判', 'いま', '裁判員裁判', '重大'])
32 : 0.9994680285453796 -- ['公開講座開催案内2020年度一橋大学公開講座', '裁判員裁判', 'いま', '裁判員裁判', '重大'])
23 : 0.9994110465049744 -- ['学業優秀学生(在学生)', '決定', 'する', '2021年4月9日', '令和2年度一年間'])
